In [1]:
from init import *

In [2]:
def BuildModel_Simple():
    m = Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(IndependentGaussian())
    m.Add(BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [3]:
m = BuildModel_Simple()
m.evaluations.append(EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentGaussian", "refs": []}, {"name": "BrownianMotion", "start": 0.1, "args": [0.2, 2]}], "evaluations": [{"state": 0, "time": 2, "histograms": []}], "RandomSeed": -1, "RunTimeoutSeconds": 1, "MemoryLimitKB": 64}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'histograms': [],
 'mean': [0.49353769421577454],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'run_time_seconds': {'mean': 0.0002956379612442106,
  'n': 768,
  'skewness': None,
  'stddev': 7.816727816134517e-07},
 'skewness': [0.008217498660087585],
 'stddev': [2.8386354446411133],
 'time_points': [2.0],
 'time_steps': [10]}

In [5]:
import time
m = BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 1000
m.RunTimeoutSeconds = 16.7
for t in [1,2,5,10]:
    m.evaluations.append(EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'histograms': [],
 'mean': [0.29631662368774414,
  0.4970751106739044,
  1.0937957763671875,
  2.093475341796875],
 'names': ['BrownianMotion'],
 'npaths': [1000000, 1000000, 1000000, 1000000],
 'run_time_seconds': {'mean': 1.5370986461639404,
  'n': 768,
  'skewness': None,
  'stddev': 0.0012977849692106247},
 'skewness': [-0.0011737046297639608,
  -0.0006328717572614551,
  -0.0028392188251018524,
  0.0019566339906305075],
 'stddev': [2.0003013610839844,
  2.8279387950897217,
  4.473629951477051,
  6.327223777770996],
 'time_points': [1.0, 2.0, 5.0, 10.0],
 'time_steps': [100, 200, 500, 1000]}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

0.7 nanoseconds to make a single time step


# Two processes, Three evaluations

In [7]:
def TwoProcessesModel ():
    m = Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(IndependentGaussian())
    m.Add(BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    m.Add(BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [8]:
model = Model()
model.TimeStart = 0
model.TimeSteps = 10
model.NumPaths = 10000
model.Add(IndependentGaussian())
model.Add(BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.Add(BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.TimeSteps = 10000
for t in [1,2,5]:
    model.evaluations.append(EvaluationPoint(state=0,time=t))
r = requests.post(f'{server}/model',model.json())
r.json()

{'histograms': [],
 'mean': [0.30820029973983765,
  0.3099619150161743,
  0.47641634941101074,
  0.5191692113876343,
  1.0786408185958862,
  1.1569509506225586],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'run_time_seconds': {'mean': 0.27971744537353516,
  'n': 768,
  'skewness': None,
  'stddev': 4.760056617669761e-05},
 'skewness': [-0.018459312617778778,
  0.012697786092758179,
  0.011788311414420605,
  0.05272487550973892,
  -0.023920131847262383,
  0.007317312527447939],
 'stddev': [1.9790846109390259,
  1.9848570823669434,
  2.8233115673065186,
  2.806223154067993,
  4.440822601318359,
  4.439358234405518],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [2000, 4000, 10000]}

In [9]:
er = EvaluationResults(r.json(),model)
er

2 states with 3 evaluations

In [10]:
er.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,1.0,2000,10000,0.308200,0.019791,1.979085,-0.018459
1,BrownianMotion,,1,1.0,2000,10000,0.309962,0.019849,1.984857,0.012698
2,BrownianMotion,,0,2.0,4000,10000,0.476416,0.028233,2.823312,0.011788
3,BrownianMotion,,1,2.0,4000,10000,0.519169,0.028062,2.806223,0.052725
4,BrownianMotion,,0,5.0,10000,10000,1.078641,0.044408,4.440823,-0.023920
5,BrownianMotion,,1,5.0,10000,10000,1.156951,0.044394,4.439358,0.007317
